# 2. Stereo Vision Calibration

Reads the image pairs saved by `1.Acquiring_Calibration_Images.ipynb` from
`stereoLeft/` and `stereoRight/`, detects checkerboard corners in each pair,
and runs OpenCV stereo calibration via the StereoVision library.

Outputs intrinsics and extrinsics to `calib_result/` for use in
`3.Depthmap_with_Tuning_Bar.ipynb`.

**This notebook has no camera dependency** — all processing is offline.

---
**Camera: Lucid PHD IMX273 — 40 mm baseline, 3 mm FL**  
**Acquisition mode: 2:1 sensor binning (average) → 720 × 540 per sensor**

The baseline distance is not set here; it is computed from the calibration
images and stored in the exported extrinsics (translation vector `T`).

## Dependencies

In [ ]:
!pip install cython==3.0.11
!pip install numpy==1.21.6
!pip install StereoVision==1.0.4
!pip install glob2==0.7
!pip install opencv-python==4.5.3.56

In [ ]:
import glob
import os
import cv2
import numpy as np
from stereovision.calibration import StereoCalibrator, StereoCalibration
from stereovision.exceptions import ChessboardNotFoundError

## Configuration

Calibration images are captured at the native binned resolution (720 × 540),
so no additional downsampling is needed. The `image_size` below matches the
saved images exactly; the `cv2.resize` call in the loop is kept as a no-op
safety net in case images were captured at a different resolution.

If you used a different checkerboard, update `rows`, `columns`, and
`square_size` to match.

In [ ]:
# Checkerboard inner-corner count and physical square size (cm).
# Defaults match the supplied 9×6 / 25 mm target (9x6_1-8cm_chessboard.png).
rows        = 6
columns     = 9
square_size = 2.5  # cm

# Native binned resolution per sensor (1440×1080 with 2:1 average binning).
image_size = (720, 540)

## Corner detection pass

Each image pair is resized, then the StereoVision library searches for
checkerboard corners. Pairs where the board is not fully visible in both
frames are automatically discarded.

In [ ]:
calibrator    = StereoCalibrator(rows, columns, square_size, image_size)
photo_counter = 0
error_counter = 0

# Create calib_result directory if it does not exist.
calib_result_path = os.path.join(os.getcwd(), "calib_result")
os.makedirs(calib_result_path, exist_ok=True)

# os.path.join keeps paths cross-platform (original used Windows backslash literals).
left_glob  = os.path.join(os.getcwd(), "stereoLeft",  "*.png")
right_glob = os.path.join(os.getcwd(), "stereoRight", "*.png")

images_left  = sorted(glob.glob(left_glob))
images_right = sorted(glob.glob(right_glob))

if not images_left:
    raise FileNotFoundError(
        f"No images found in {left_glob}. "
        "Run 1.Acquiring_Calibration_Images.ipynb first."
    )

print(f"Found {len(images_left)} left / {len(images_right)} right images.")
print("Starting corner detection...\n")

for img_left_path, img_right_path in zip(images_left, images_right):
    print(f"Pair {photo_counter:03d}  {os.path.basename(img_left_path)}")

    img_left  = cv2.imread(img_left_path,  cv2.IMREAD_UNCHANGED)
    img_right = cv2.imread(img_right_path, cv2.IMREAD_UNCHANGED)

    # Ensure both images are BGR (StereoVision expects colour input).
    if img_left.ndim == 2:
        img_left  = cv2.cvtColor(img_left,  cv2.COLOR_GRAY2BGR)
        img_right = cv2.cvtColor(img_right, cv2.COLOR_GRAY2BGR)

    img_left  = cv2.resize(img_left,  image_size)
    img_right = cv2.resize(img_right, image_size)

    try:
        calibrator._get_corners(img_left)
        calibrator._get_corners(img_right)
    except ChessboardNotFoundError as e:
        print(f"  → discarded: {e}")
        error_counter += 1
    else:
        # add_corners also calls cv2.imshow to preview detected corners.
        calibrator.add_corners((img_left, img_right), True)

    photo_counter += 1

print(f"\nDone. {photo_counter - error_counter} pairs accepted, {error_counter} discarded.")

## Stereo calibration

Runs `cv2.stereoCalibrate` under the hood and exports the result.
A reprojection error below **0.5 px** indicates a good calibration.

> **Note on distortion:** the Edmund Optics #20-061 lens has 34.78% barrel
> distortion at full field. If the reprojection error is high, consider
> enabling rational-model flags in a custom calibration pass
> (`cv2.CALIB_RATIONAL_MODEL`) which uses a 8-coefficient distortion model
> instead of the default 5.

In [ ]:
print("Starting calibration — this may take a few minutes...")
calibration = calibrator.calibrate_cameras()
calibration.export(calib_result_path)
print("Calibration complete!")

avg_error = calibrator.check_calibration(calibration)
print(f"Average reprojection error: {avg_error:.4f} px")
if avg_error > 0.5:
    print("  ⚠ Error > 0.5 px — consider re-capturing images from more angles.")
else:
    print("  ✓ Error within acceptable range.")

## Inspect rectified output

Load the exported calibration and apply it to the last image pair.
Corresponding points should fall on the same horizontal scanline in both
rectified images. If they don't, re-capture calibration images.

In [ ]:
calibration   = StereoCalibration(input_folder=calib_result_path)
rectified_pair = calibration.rectify((img_left, img_right))

cv2.namedWindow("Left  — rectified",  cv2.WINDOW_NORMAL)
cv2.namedWindow("Right — rectified", cv2.WINDOW_NORMAL)
cv2.imshow("Left  — rectified",  rectified_pair[0])
cv2.imshow("Right — rectified", rectified_pair[1])

cv2.waitKey(0)
cv2.destroyAllWindows()

print("Open 3.Depthmap_with_Tuning_Bar.ipynb to continue.")